<a href="https://colab.research.google.com/github/armandoordonez/GenAI/blob/main/RAG_Chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install chromadb sentence-transformers transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s e

In [ ]:
# Ejemplo de datos de entrada
documents = [
    "La inteligencia artificial es una rama de la informática.",
    "Los modelos de lenguaje generativo son muy populares en IA.",
    "ChromaDB es una base de datos vectorial eficiente.",
    "Python es un lenguaje de programación ampliamente utilizado."
]

In [ ]:
from sentence_transformers import SentenceTransformer

# Cargar un modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings para los documentos
embeddings = model.encode(documents)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import chromadb

# Crear una instancia de ChromaDB con persistencia
client = chromadb.PersistentClient(path="chroma_db")  # Usa "PersistentClient" para almacenar datos

# Crear una colección
collection = client.create_collection(name="knowledge_base")

# Almacenar los embeddings y documentos en la colección
for i, (doc, emb) in enumerate(zip(documents, embeddings)):
    collection.add(
        ids=[str(i)],  # Identificador único para cada documento
        embeddings=[emb.tolist()],  # Convertir el embedding a lista
        documents=[doc]  # Texto del documento
    )

In [ ]:
def retrieve(query, top_k=2):
    # Generar el embedding de la consulta
    query_embedding = model.encode(query)

    # Buscar los documentos más similares
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=top_k
    )

    return results['documents'][0]  # Devolver los documentos recuperados

In [ ]:
from transformers import pipeline

# Cargar un modelo de lenguaje alternativo
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-125M")

def generate_response(query, retrieved_docs):
    # Combinar los documentos recuperados en un contexto
    context = " ".join(retrieved_docs)

    # Formular la entrada para el modelo
    prompt = f"Contexto: {context}\nPregunta: {query}\nRespuesta:"

    # Generar la respuesta con truncamiento explícito
    response = generator(
        prompt,
        max_length=50,
        num_return_sequences=1,
        truncation=True  # Habilitar truncamiento
    )
    return response[0]['generated_text']

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def rag_system(query):
    # Recuperar documentos relevantes
    retrieved_docs = retrieve(query)

    # Generar una respuesta
    response = generate_response(query, retrieved_docs)

    return response

# Ejemplo de uso
query = "¿Qué es Python"
response = rag_system(query)
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Contexto: Python es un lenguaje de programación ampliamente utilizado. La inteligencia artificial es una rama de la informática.
Pregunta: ¿Qué es Python
Respuesta:

A:

Pregunta: ¿Qué es Python
Respuesta:

Python es un lenguaje de programación ampliamente utilizado. La inteligencia artificial
